# Bash 链
本文档展示如何使用 LLM 和 bash 进程执行简单的文件系统命令。

In [ ]:
from langchain_experimental.llm_bash.base import LLMBashChain
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

text = "请编写一个在控制台打印'Hello World'的 bash 脚本。"

bash_chain = LLMBashChain.from_llm(llm, verbose=True)

bash_chain.invoke(text)



> Entering new LLMBashChain chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
echo "Hello World"
```
Code: ['echo "Hello World"']
Answer: Hello World

> Finished chain.


'Hello World\n'

## 自定义提示
您也可以自定义使用的提示。这里是一个避免使用'echo'命令的示例

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain_experimental.llm_bash.prompt import BashOutputParser

_PROMPT_TEMPLATE = """如果有人要求您执行任务，您的工作是提供一系列能够完成该任务的 bash 命令。不需要在答案中包含"#!/bin/bash"。请确保按步骤进行推理，使用以下格式：
问题："将名为'target'的目录中的文件复制到与target同级的新目录'myNewDirectory'中"
我需要执行以下操作：
- 列出目录中的所有文件
- 创建新目录
- 将第一个目录中的文件复制到第二个目录
```bash
ls
mkdir myNewDirectory
cp -r target/* myNewDirectory
```

编写脚本时请不要使用'echo'。

这就是格式。开始！
问题：{question}"""

PROMPT = PromptTemplate(
    input_variables=["question"],
    template=_PROMPT_TEMPLATE,
    output_parser=BashOutputParser(),
)

In [ ]:
bash_chain = LLMBashChain.from_llm(llm, prompt=PROMPT, verbose=True)

text = "请编写一个在控制台打印'Hello World'的 bash 脚本。"

bash_chain.invoke(text)



> Entering new LLMBashChain chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
printf "Hello World\n"
```
Code: ['printf "Hello World\\n"']
Answer: Hello World

> Finished chain.


'Hello World\n'

## 持久化终端

默认情况下，每次调用链时都会在单独的子进程中运行。通过实例化持久化的 bash 进程可以改变这个行为。

In [ ]:
from langchain_experimental.llm_bash.bash import BashProcess

persistent_process = BashProcess(persistent=True)
bash_chain = LLMBashChain.from_llm(llm, bash_process=persistent_process, verbose=True)

text = "列出当前目录然后向上移动一级。"

bash_chain.invoke(text)



> Entering new LLMBashChain chain...
List the current directory then move up a level.

```bash
ls
cd ..
```
Code: ['ls', 'cd ..']
Answer: cpal.ipynb  llm_bash.ipynb  llm_symbolic_math.ipynb
index.mdx   llm_math.ipynb  pal.ipynb
> Finished chain.


'cpal.ipynb  llm_bash.ipynb  llm_symbolic_math.ipynb\r\nindex.mdx   llm_math.ipynb  pal.ipynb'

In [ ]:
# 再次运行相同的命令，可以看到状态在多次调用之间得以保持
bash_chain.invoke(text)



> Entering new LLMBashChain chain...
List the current directory then move up a level.

```bash
ls
cd ..
```
Code: ['ls', 'cd ..']
Answer: _category_.yml	data_generation.ipynb		   self_check
agents		graph
code_writing	learned_prompt_optimization.ipynb
> Finished chain.


'_category_.yml\tdata_generation.ipynb\t\t   self_check\r\nagents\t\tgraph\r\ncode_writing\tlearned_prompt_optimization.ipynb'